In [74]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
# import seaborn as sns
# from matplotlib import pyplot as plt
# %matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

from pycaret.classification import * 

# pd.options.display.max_columns = 5000
# pd.options.display.max_rows = 5000

injured_pred = 'injured_in_26_week'

simple_features = ['Height', 'Weight', 'age','cum_injury_total', 'weeks_since_last_injury', 'Min_cum','Serie A_cum',
 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum',
 'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum']

extended_features = ['Height', 'Weight', 'defender', 'attacker', 'midfielder', 'goalkeeper', 'right_foot', 'age', 'cum_injury_total', 'weeks_since_last_injury', 'Min_cum', 'Gls_cum', 'Ast_cum', 'PK_cum', 'PKatt_cum',
 'Sh_cum', 'SoT_cum', 'CrdY_cum', 'CrdR_cum', 'Touches_cum', 'Press_cum', 'Tkl_cum', 'Int_cum', 'Blocks_cum', 'xG_cum', 'npxG_cum', 'xA_cum', 'SCA_cum', 'GCA_cum', 'Cmp_cum',
 'Att_cum', 'Prog_cum', 'Carries_cum', 'Prog.1_cum', 'Succ_cum', 'Att.1_cum', 'Fls_cum', 'Fld_cum', 'Off_cum', 'Crs_cum', 'TklW_cum', 'OG_cum', 'PKwon_cum','PKcon_cum', 'Serie A_cum',
 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum',
 'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum']

In [39]:
# Load Dataset

# Renzo's Path
# dataset = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df_5.csv')
#
# Sravan's Path
# dataset = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv')

# Sravan's Parquet Path
dataset = pd.read_parquet('dataframes_blog/dataset_for_model_final.parquet') 

In [40]:
# dataset[dataset['FBRefID']=='dea698d9'].to_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/ronaldo.csv', index=False)
dataset = dataset[dataset['drop'] == 0]
dataset = dataset[dataset['goalkeeper'] == 0]

In [41]:
# Get Train Test Split
df_train = dataset[dataset['cum_week'] <= dataset["train_split"]].dropna()
df_test = dataset[dataset['cum_week'] > dataset["train_split"]].dropna()

In [57]:
X_train = df_train[extended_features]
y_train = df_train[injured_pred]

X_test = df_test[extended_features]
y_test = df_test[injured_pred]

In [6]:
exp_clf = setup(dataset[extended_features + [injured_pred]], target=injured_pred, fix_imbalance=True, feature_selection=True, fold=2)

,Description,Value
0,session_id,8301
1,Target,injured_in_26_week
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(1604775, 59)"
5,Missing Values,True
6,Numeric Features,53
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [15]:
set_config('X_train', X_train)
set_config('X_test', X_test)
set_config('y_train', y_train)
set_config('y_test', y_test)

In [ ]:
best_model = compare_models()

IntProgress(value=0, description='Processing: ', max=74)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:14:53
Status,. . . . . . . . . . . . . . . . . .,Fitting 2 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9127,0.5663,0.0243,0.0923,0.0385,0.0088,0.0110,23.2150
ridge,Ridge Classifier,0.8370,0.0000,0.2233,0.1301,0.1645,0.0811,0.0847,4.2600
dt,Decision Tree Classifier,0.7823,0.5104,0.1951,0.0806,0.1140,0.0138,0.0156,20.3300
ada,Ada Boost Classifier,0.7520,0.6318,0.3903,0.1207,0.1843,0.0839,0.1041,79.4750
knn,K Neighbors Classifier,0.7232,0.5384,0.2958,0.0859,0.1331,0.0245,0.0312,470.3950
qda,Quadratic Discriminant Analysis,0.6751,0.5681,0.3238,0.0830,0.0917,0.0100,0.0172,7.0500
lr,Logistic Regression,0.6738,0.5990,0.4502,0.1013,0.1654,0.0546,0.0783,99.0400
svm,SVM - Linear Kernel,0.5622,0.0000,0.4735,0.1168,0.1173,0.0311,0.0457,95.9150
nb,Naive Bayes,0.4668,0.5831,0.6314,0.0874,0.1506,0.0293,0.0477,4.8900


In [9]:
# df = pull()
# df.to_csv('results_1_year.csv', index=False)
df = pull().sort_values(by=['F1'], ascending=False)

In [10]:
df

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.7520,0.6318,0.3903,0.1207,0.1843,0.0839,0.1041,79.340
gbc,Gradient Boosting Classifier,0.8345,0.6419,0.2455,0.1367,0.1756,0.0919,0.0967,302.775
lr,Logistic Regression,0.6738,0.5990,0.4502,0.1013,0.1654,0.0546,0.0783,122.505
ridge,Ridge Classifier,0.8370,0.0000,0.2233,0.1301,0.1645,0.0811,0.0847,4.340
lda,Linear Discriminant Analysis,0.8370,0.6136,0.2233,0.1301,0.1644,0.0811,0.0847,8.845
nb,Naive Bayes,0.4668,0.5831,0.6314,0.0874,0.1506,0.0293,0.0477,4.845
knn,K Neighbors Classifier,0.7232,0.5384,0.2958,0.0859,0.1331,0.0245,0.0312,509.850
svm,SVM - Linear Kernel,0.5622,0.0000,0.4735,0.1168,0.1173,0.0311,0.0457,96.580
dt,Decision Tree Classifier,0.7823,0.5104,0.1951,0.0806,0.1140,0.0138,0.0156,20.330
qda,Quadratic Discriminant Analysis,0.6751,0.5681,0.3238,0.0830,0.0917,0.0100,0.0172,7.375


In [12]:
# Select best model
model = create_model(df.index[0], fold=5)
save_model(model, 'model_1_semester')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7359,0.6265,0.4045,0.1160,0.1803,0.0773,0.0990
1,0.7256,0.6360,0.4322,0.1173,0.1845,0.0806,0.1056
2,0.7545,0.6208,0.3624,0.1153,0.1749,0.0740,0.0909
3,0.7362,0.6308,0.4039,0.1160,0.1802,0.0773,0.0988
4,0.7345,0.6389,0.4006,0.1145,0.1781,0.0748,0.0958
Mean,0.7373,0.6306,0.4007,0.1158,0.1796,0.0768,0.0980
SD,0.0094,0.0065,0.0223,0.0009,0.0031,0.0023,0.0048


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='injured_in_26_week',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 n...
                  Advanced_Feature_Selection_Classic(ml_usecase='classification',
                                                     n_jobs=-1,
                                                     random_state=8301,
                 

In [14]:
tuned_model = tune_model(model, optimize = 'F1')
save_model(tuned_model, 'model_1_semester_tuned')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7363,0.6396,0.4127,0.1180,0.1835,0.0809,0.1036
1,0.7292,0.6422,0.4297,0.1183,0.1856,0.0822,0.1070
Mean,0.7327,0.6409,0.4212,0.1182,0.1845,0.0815,0.1053
SD,0.0036,0.0013,0.0085,0.0002,0.0010,0.0007,0.0017


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='injured_in_26_week',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 n...
                  Advanced_Feature_Selection_Classic(ml_usecase='classification',
                                                     n_jobs=-1,
                                                     random_state=8301,
                 

In [12]:
# Loading the saved model
model_1_semester_tuned = load_model('model_1_semester_tuned')

Transformation Pipeline and Model Successfully Loaded


In [13]:
model_1_semester_tuned

Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(ml_usecase='classification',
                                      target='injured_in_26_week')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least '
                                                                         'fr...
                ('clean_names', Clean_Colum_Names()),
                ('feature_select',
                 Advanced_Feature_Selection_Classic(n_jobs=-1,
                                                    random_state=8301,
                                                    subclass='binary',
                  

In [58]:
predicted_injuries = model_1_semester_tuned[23].predict(X_test)
prob_of_injury = model_1_semester_tuned[23].predict_proba(X_test)[:, 1]


In [62]:
predictions_df = X_test
predictions_df['predicted_injuries'] = predicted_injuries
predictions_df['prob_of_injury'] = prob_of_injury
predictions_df['actual_injuries'] = y_test

In [66]:
df_test = pd.merge(df_test, predictions_df[['predicted_injuries', 'prob_of_injury']], left_index=True, \
    right_index=True)

In [90]:
df_test['cum_predicted_injuries'] = df_test.groupby(by=['name'])['predicted_injuries'].cumsum()


In [92]:
df_picked_player = df_test[df_test['name'] == 'Renato Tapia']
df_picked_player['cum_injury_total_in_window'] = df_picked_player.groupby(by=['name'])['injured_in_26_week'].cumsum()

In [93]:
base = alt.Chart(df_picked_player).encode(x='cum_week:Q')
chart = alt.layer(base.mark_line(color='red').encode(y='cum_predicted_injuries:Q'), base.mark_line(color='blue'). \
        encode(y='cum_injury_total_in_window:Q')).properties(width=800, height=300)
chart

alt.LayerChart(...)

In [88]:
df_picked_player

,FBRefID,date,agg_week,agg_year,Injury,injury_week,injury_year,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Prog,Carries,Prog.1,Succ,Att.1,Fls,Fld,Off,Crs,TklW,OG,PKwon,PKcon,Won,Loss,Draw,FootAbility,release_week,was_match,Serie A,Premier League,La Liga,Ligue 1,Bundesliga,Champions Lg,Europa Lg,FIFA World Cup,UEFA Nations League,UEFA Euro,Copa América,Away,Home,Neutral,week,year,PlayerFullName,name,Position:,Height,Weight,Foot,Birth,Nationality,Photo,InternationalReputation,Twitter,Instagram,Place of birth:,Citizenship:,cum_week,defender,attacker,midfielder,goalkeeper,right_foot,left_foot,injury_count,cum_injury,age,unique_injury_count,cum_injury_total,previous_injury_week,weeks_since_last_injury,injured,injured_in_1_week,injured_in_4_week,injured_in_12_week,injured_in_26_week,injured_in_52_week,injury_count_in_1_week,injury_count_in_4_week,injury_count_in_12_week,injury_count_in_26_week,injury_count_in_52_week,cum_injury_in_1_week,cum_injury_in_4_week,cum_injury_in_12_week,cum_injury_in_26_week,cum_injury_in_52_week,cum_sum,Min_cum,Gls_cum,Ast_cum,PK_cum,PKatt_cum,Sh_cum,SoT_cum,CrdY_cum,CrdR_cum,Touches_cum,Press_cum,Tkl_cum,Int_cum,Blocks_cum,xG_cum,npxG_cum,xA_cum,SCA_cum,GCA_cum,Cmp_cum,Att_cum,Prog_cum,Carries_cum,Prog.1_cum,Succ_cum,Att.1_cum,Fls_cum,Fld_cum,Off_cum,Crs_cum,TklW_cum,OG_cum,PKwon_cum,PKcon_cum,Won_cum,Loss_cum,Draw_cum,was_match_cum,Serie A_cum,Premier League_cum,La Liga_cum,Ligue 1_cum,Bundesliga_cum,Champions Lg_cum,Europa Lg_cum,FIFA World Cup_cum,UEFA Nations League_cum,UEFA Euro_cum,Copa América_cum,Away_cum,Home_cum,Neutral_cum,defender_cum,attacker_cum,midfielder_cum,goalkeeper_cum,right_foot_cum,left_foot_cum,drop,last_week,train_split,predicted_injuries,prob_of_injury,cum_predicted_injuries
390098,34269bdf,2019-12-08,48.0,2019.0,0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,96.0,4038.0,Renato Tapia Cortijo,Renato Tapia,midfield - Defensive Midfield,185.0,78.0,RIGHT,"July 28, 1995",Peru,https://fbref.com/req/202005121/images/headsho...,2016-17 Dutch Eredivisie Champion,0,0,Lima,Peru Netherlands,320.0,0,0,1,0,1,0,0.0,36.0,24.0,0.0,8.0,286.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,36.0,36.0,36.0,36.0,31825.0,2786.0,1.0,2.0,0.0,0.0,25.0,8.0,7.0,0.0,194.0,71.0,9.0,25.0,6.0,0.1,0.1,0.0,4.0,0.0,118.0,146.0,9.0,131.0,9.0,2.0,4.0,37.0,33.0,2.0,3.0,34.0,1.0,0.0,0.0,15.0,14.0,13.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,6.0,16.0,19.0,7.0,0,0,35,0,35,0,0.0,425.0,319.0,1.0,0.502611,1.0
390099,34269bdf,2019-12-15,49.0,2019.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49.0,2019.0,Renato Tapia Cortijo,Renato Tapia,midfield - Defensive Midfield,185.0,78.0,RIGHT,"July 28, 1995",Peru,https://fbref.com/req/202005121/images/headsho...,2016-17 Dutch Eredivisie Champion,0,0,Lima,Peru Netherlands,321.0,0,0,1,0,1,0,0.0,36.0,24.0,0.0,8.0,286.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,36.0,36.0,36.0,36.0,31825.0,2786.0,1.0,2.0,0.0,0.0,25.0,8.0,7.0,0.0,194.0,71.0,9.0,25.0,6.0,0.1,0.1,0.0,4.0,0.0,118.0,146.0,9.0,131.0,9.0,2.0,4.0,37.0,33.0,2.0,3.0,34.0,1.0,0.0,0.0,15.0,14.0,14.0,43.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,6.0,17.0,19.0,7.0,0,0,36,0,36,0,0.0,425.0,319.0,1.0,0.502611,2.0
390100,34269bdf,2019-12-22,50.0,2019.0,0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,12.0,1.0,12.0,2.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0,4038.0,Renato Tapia Cortijo,Renato Tapia,midfield - Defensive Midfield,185.0,78.0,RIGHT,"July 28, 1995",Peru,https://fbref.com/req/2